In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os

In [2]:
categories=os.listdir('C:/Users/ishan/Face-Mask-recognition/dataset')
categories

['with mask', 'without mask']

# data processing

In [3]:
image=[]
label=[]
for category in categories:
    img_path=os.path.join('C:/Users/ishan/Face-Mask-recognition/dataset',category)
    images=os.listdir(img_path)
    
    for img in images:
        data=cv2.imread(os.path.join(img_path,img))
        img_resized=cv2.resize(data,(100,100))
        image.append(img_resized)
        label.append(categories.index(category))
np.array(label)      

array([0, 0, 0, ..., 1, 1, 1])

In [4]:
image[0].shape

(100, 100, 3)

In [5]:
images=np.reshape(image,(len(image),100,100,3))      
images.shape

(1376, 100, 100, 3)

In [6]:
labels=np.reshape(label,(len(label),))
labels.shape

(1376,)

In [7]:
labels

array([0, 0, 0, ..., 1, 1, 1])

In [8]:
from sklearn.utils import shuffle

images,labels=shuffle(images,labels)            

# spliting training and testing data

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
X_train,X_test,y_train,y_test=train_test_split(images,labels,test_size=0.2,random_state=42)

In [11]:
X_train.shape

(1100, 100, 100, 3)

In [12]:
from keras.utils import to_categorical

y_train=to_categorical(y_train)
y_test=to_categorical(y_test)
y_test

Using TensorFlow backend.


array([[0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.

# Convolutional Neural Network Model

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,BatchNormalization,Dense,Dropout,Input,Flatten

In [14]:
model=Sequential()

model.add(Input(shape=(100,100,3)))

model.add(Conv2D(100,3,activation='relu',padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.5))

model.add(Conv2D(200,3,activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(2,activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 100, 100, 100)     2800      
_________________________________________________________________
batch_normalization (BatchNo (None, 100, 100, 100)     400       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 50, 50, 100)       0         
_________________________________________________________________
dropout (Dropout)            (None, 50, 50, 100)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 48, 48, 200)       180200    
_________________________________________________________________
batch_normalization_1 (Batch (None, 48, 48, 200)       800       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 24, 24, 200)       0

# Training the model

In [15]:
from keras.callbacks import ModelCheckpoint

In [16]:
checkpoint=ModelCheckpoint('model-{epoch:03d}.model',monitor='val_accuracy',save_best_only=True)   

In [17]:
history=model.fit(X_train,y_train,epochs=30,validation_split=0.1,verbose=1,callbacks=[checkpoint])

Train on 990 samples, validate on 110 samples
Epoch 1/30
960/990 [============================>.] - ETA: 2s - loss: 2.5904 - accuracy: 0.8385WARNING:tensorflow:From C:\Users\ishan\.conda\envs\tensorfloww\lib\site-packages\tensorflow_core\python\ops\resource_variable_ops.py:1786: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: model-001.model\assets
990/990 [==============================] - 94s 95ms/sample - loss: 2.5270 - accuracy: 0.8414 - val_loss: 6.6712 - val_accuracy: 0.7727
Epoch 2/30
990/990 [==============================] - 93s 94ms/sample - loss: 0.4711 - accuracy: 0.9606 - val_loss: 0.2583 - val_accuracy: 0.9727
Epoch 3/30
990/990 [==============================] - 79s 80ms/sample - loss: 0.2394 - accuracy: 0.9758 - val_loss: 0.6469 - val_accuracy: 0

In [18]:
_,accuracy=model.evaluate(X_test,y_test)

276/276 [==============================] - 2s 6ms/sample - loss: 4.3473 - accuracy: 0.8768


In [19]:
pred=model.predict(X_test)
pred

array([[5.79858759e-22, 1.00000000e+00],
       [4.83044784e-16, 1.00000000e+00],
       [1.53490709e-09, 1.00000000e+00],
       [9.99999881e-01, 1.00336081e-07],
       [1.00000000e+00, 3.73200996e-17],
       [1.00000000e+00, 1.99811515e-21],
       [1.20556503e-27, 1.00000000e+00],
       [8.84193341e-09, 1.00000000e+00],
       [5.81702488e-28, 1.00000000e+00],
       [5.23642171e-25, 1.00000000e+00],
       [1.00000000e+00, 1.99315608e-09],
       [0.00000000e+00, 1.00000000e+00],
       [9.99944687e-01, 5.53171121e-05],
       [9.98138905e-01, 1.86109031e-03],
       [4.20423388e-23, 1.00000000e+00],
       [2.01403410e-08, 1.00000000e+00],
       [1.00000000e+00, 2.49476178e-28],
       [1.18832122e-17, 1.00000000e+00],
       [0.00000000e+00, 1.00000000e+00],
       [0.00000000e+00, 1.00000000e+00],
       [1.00000000e+00, 9.12894068e-28],
       [1.53343767e-32, 1.00000000e+00],
       [2.27463329e-31, 1.00000000e+00],
       [1.00000000e+00, 4.59146259e-26],
       [1.000000

In [20]:
np.argmax(pred[45])

1

# Detection of face mask

In [29]:
face_detection=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')    

In [30]:
labels_dict={0:'WITH MASK',1:'WITHOUT MASK'}
def mask_detection(img):
    
    faces=face_detection.detectMultiScale(img)
    for (x,y,w,h) in faces:
        face_img=img[y:y+h,x:x+w]                    
        face_img=cv2.resize(face_img,(100,100))
        face_img=np.reshape(face_img,(1,100,100,3))
        predict=model.predict(face_img)
        
        if np.argmax(predict[0])==0:
            cv2.rectangle(img,(x,y),(x+w,y+h),color=(0,255,0),thickness=2)
            cv2.putText(img, labels_dict[0], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.6,(255,255,255),2)
        elif np.argmax(predict[0])==1:
            cv2.rectangle(img,(x,y),(x+w,y+h),color=(0,0,255),thickness=2)
            cv2.putText(img, labels_dict[1], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.6,(255,255,255),2)
    return faces

In [32]:
cap=cv2.VideoCapture(0)

while True:
    ret,frame=cap.read()

    mask_detection(frame)
    
    cv2.imshow('Live Video Mask Detection',frame)
    
    if cv2.waitKey(3)==27:
        break
        
cap.release()
cv2.destroyAllWindows()